# 关键概念

欢迎来到``brainstate``的世界！

本节简要介绍了 ``BrainState`` 框架的一些关键概念。

``BrainState`` 是一个专门面向脑动力学建模的高性能计算框架，基于[JAX](https://github.com/jax-ml/jax) 构建。它为神经科学研究人员、计算神经科学工作者、以及类脑计算研究者提供了一套完整的工具链，用于构建、优化和部署各类神经网络模型。它整合了现代硬件加速、自动微分、事件驱动计算等先进特性，专为神经网络尤其是脉冲神经网络（Spiking Neural Networks, SNN）设计。以下教程将详细介绍其核心功能及其使用场景，帮助您快速上手并理解如何使用 BrainState 构建和优化脑动力学模型。


In [17]:
import jax.numpy as jnp

import brainstate

## 核心功能概览

``BrainState`` 的主要功能包括以下几个部分：

- **程序编译**： 支持通过 [State](../apis/brainstate.rst) 语法进行程序[编译](../apis/compile.rst)，可在 CPU、GPU、TPU 等不同硬件设备上部署计算模型。
- **程序功能增强**： 提供 [PyGraph](../apis/graph.rst) 语法的[增强](../apis/augment.rst)功能，通过自动微分、批处理等机制简化构建复杂计算模型的过程。
- **事件驱动计算**： 支持基于 [事件驱动计算](../apis/event.rst) 的算子优化，大幅提升脉冲神经网络的效率和可扩展性。
- **其它附加功能**： 包括随机数生成、梯度代理、模型参数管理等多个便捷的辅助工具，方便用户进行模型搭建。

接下来，我们将逐项深入探讨这些功能的使用方法和优化策略。

## 1. ``State`` 语法

JAX 的程序编写方式通常是通过函数式编程实现，但对于脑动力学模型等复杂计算任务，这种方式可能显得不够直观。``BrainState`` 提供了 ``State`` 语法，一种高度抽象的接口，帮助用户更直观地定义和管理计算状态。``State`` 语法的核心特性包括：

- 所有需要改变的量都被封装在 ``State`` 对象中，方便用户追踪和调试模型状态。
- 其它没有被 ``State`` 封装的变量都是不可变的，在程序编译后不能再被修改。brainstate中提供的编译函数可以在[``brainstate.compile`` 模块](../apis/compile.rst)中查看。

这意味着，在BrainState中，所有需要改变的变量都应该被封装在 ``State`` 对象中，以确保程序的正确性和可维护性。

``State`` 可以有不同的子类，比如，在brainstate中，``ParamState`` 是 ``State`` 的一个子类，用于封装模型参数；``RandomState`` 是 ``State`` 的另一个子类，用于封装随机数生成器的状态。用户可以轻松扩展自己的 ``State`` 子类，以满足不同的需求。比如：

In [18]:
class Counter(brainstate.State):
    pass

在上面的例子中，通过继承 ``State`` 类，我们定义了一个 ``Counter`` 类，它可以用于封装计数器的状态。这种方式使得用户可以更灵活地定义和管理模型的状态，提高了代码的可读性和可维护性。

``State`` 可以wrap任意的Python数据，比如整数、浮点数、数组、``jax.Array``等，以及封装在字典或者数组中的上述任意Python数据。用户可以通过 ``State.value`` 属性来访问和修改这些数据。比如：

In [19]:
example = brainstate.State(jnp.ones(3))

example

State(
  value=ShapedArray(float32[3])
)

In [20]:
example.value = brainstate.random.random(3)

example

State(
  value=ShapedArray(float32[3])
)

``State`` 支持任意 [PyTree](https://jax.readthedocs.io/en/latest/working-with-pytrees.html)，这意味着用户可以将任意的数据结构封装在 ``State`` 对象中，方便地进行状态管理和计算。

In [21]:
example2 = brainstate.State({'a': jnp.ones(3), 'b': jnp.zeros(4)})

example2

State(
  value={
    'a': ShapedArray(float32[3]),
    'b': ShapedArray(float32[4])
  }
)

## 2. ``PyGraph`` 语法

在 JAX 中，pytree（Python tree）是一种通用的数据结构，用于灵活地表示嵌套的、树状的 Python 容器。它可以包含诸如列表、元组、字典等多种容器，同时还能够嵌套不同类型的数据结构，如 NumPy 数组、JAX 数组或自定义对象。这种灵活性使得 pytree 在数据处理和模型构建中非常有用，但在科学计算的复杂场景下，它的表达能力可能受到限制。

在许多科学计算中，我们常常需要定义复杂的计算图，这些图可能包括循环引用、嵌套结构以及动态生成的计算流程，而这些情况是 pytree 结构所难以表达的。为了应对这一挑战，``brainstate`` 提供了 ``PyGraph`` 数据结构，它为用户提供了一种更直观和灵活的方式来定义和操作 Python 中各种模块化对象交织的复杂计算模型。

``PyGraph`` 的设计来自于 Flax 的 [nnx模块](https://flax.readthedocs.io/)，并在此基础上进行了扩展和优化，使其适用于 ``brainstate`` 的``State``索引、管理和操作。``PyGraph`` 由 ``brainstate.graph.Node`` 作为基础子节点构成，这些节点可以形成有向无环图（DAG），支持节点之间的循环引用，使得构建复杂计算流程变得更加自然。

以下是一个简单的代码示例。

In [22]:
class Linear(brainstate.graph.Node):
    def __init__(self, din: int, dout: int):
        self.din, self.dout = din, dout
        self.w = brainstate.ParamState(brainstate.random.rand(din, dout))
        self.b = brainstate.ParamState(jnp.zeros((dout,)))

    def __call__(self, x):
        return x @ self.w.value + self.b.value

In [23]:
model = Linear(2, 5)

model

Linear(
  din=2,
  dout=5,
  w=ParamState(
    value=ShapedArray(float32[2,5])
  ),
  b=ParamState(
    value=ShapedArray(float32[5])
  )
)

我们可以在模型中添加一个自引用形成循环图。即便如此，PyGraph 依然能正确处理这一自引用。

In [24]:
model.self = model

model

Linear(
  din=2,
  dout=5,
  w=ParamState(
    value=ShapedArray(float32[2,5])
  ),
  b=ParamState(
    value=ShapedArray(float32[5])
  ),
  self=Linear(...)
)

``brainstate.graph.Node``可以在嵌套结构中自由组合，包括任何（嵌套）pytree 类型，例如list、dict、tuple等等。以下是一个MLP的程序示例。

In [25]:
class MLP(brainstate.graph.Node):
    def __init__(self, din: int, dmid: int, dout: int, n_layer: int = 3):
        self.input = brainstate.nn.Linear(din, dmid)
        self.layers = [brainstate.nn.Linear(dmid, dmid) for _ in range(n_layer)]
        self.output = brainstate.nn.Linear(dmid, dout)

    def __call__(self, x):
        x = brainstate.functional.relu(self.input(x))
        for layer in self.layers:
            x = brainstate.functional.relu(layer(x))
        return self.output(x)


model = MLP(2, 1, 3)

model

MLP(
  input=Linear(
    in_size=(2,),
    out_size=(1,),
    w_mask=None,
    weight=ParamState(
      value={
        'bias': ShapedArray(float32[1]),
        'weight': ShapedArray(float32[2,1])
      }
    )
  ),
  layers=[
    Linear(
      in_size=(1,),
      out_size=(1,),
      w_mask=None,
      weight=ParamState(
        value={
          'bias': ShapedArray(float32[1]),
          'weight': ShapedArray(float32[1,1])
        }
      )
    ),
    Linear(
      in_size=(1,),
      out_size=(1,),
      w_mask=None,
      weight=ParamState(
        value={
          'bias': ShapedArray(float32[1]),
          'weight': ShapedArray(float32[1,1])
        }
      )
    ),
    Linear(
      in_size=(1,),
      out_size=(1,),
      w_mask=None,
      weight=ParamState(
        value={
          'bias': ShapedArray(float32[1]),
          'weight': ShapedArray(float32[1,1])
        }
      )
    )
  ],
  output=Linear(
    in_size=(1,),
    out_size=(3,),
    w_mask=None,
    weight=ParamS

``brainstate.graph`` 模块还提供了一系列强大的工具，用于构建和操作 ``PyGraph``，包括节点的创建、连接、计算、更新等功能。这些工具允许用户以模块化的方式构建计算图，轻松地管理计算流程，从而提升模型的可读性和可维护性。例如，用户可以通过简单的 API 来添加新节点、定义节点间的依赖关系、以及动态更新节点的状态。此外，``PyGraph`` 还支持对计算图的结构化表征，有助于用户直观理解计算流程的结构与运行机制。

比如，``brainstate.graph.states`` 可以轻松获取模型中涵盖的所有``State``示例：

In [26]:
states = brainstate.graph.states(model)

states

{
  ('input', 'weight'): ParamState(
    value={
      'bias': ShapedArray(float32[1]),
      'weight': ShapedArray(float32[2,1])
    }
  ),
  ('layers', 0, 'weight'): ParamState(
    value={
      'bias': ShapedArray(float32[1]),
      'weight': ShapedArray(float32[1,1])
    }
  ),
  ('layers', 1, 'weight'): ParamState(
    value={
      'bias': ShapedArray(float32[1]),
      'weight': ShapedArray(float32[1,1])
    }
  ),
  ('layers', 2, 'weight'): ParamState(
    value={
      'bias': ShapedArray(float32[1]),
      'weight': ShapedArray(float32[1,1])
    }
  ),
  ('output', 'weight'): ParamState(
    value={
      'bias': ShapedArray(float32[3]),
      'weight': ShapedArray(float32[1,3])
    }
  )
}

In [27]:
states.to_nest()

{
  'input': {
    'weight': ParamState(
      value={
        'bias': ShapedArray(float32[1]),
        'weight': ShapedArray(float32[2,1])
      }
    )
  },
  'layers': {
    0: {
      'weight': ParamState(
        value={
          'bias': ShapedArray(float32[1]),
          'weight': ShapedArray(float32[1,1])
        }
      )
    },
    1: {
      'weight': ParamState(
        value={
          'bias': ShapedArray(float32[1]),
          'weight': ShapedArray(float32[1,1])
        }
      )
    },
    2: {
      'weight': ParamState(
        value={
          'bias': ShapedArray(float32[1]),
          'weight': ShapedArray(float32[1,1])
        }
      )
    }
  },
  'output': {
    'weight': ParamState(
      value={
        'bias': ShapedArray(float32[3]),
        'weight': ShapedArray(float32[1,3])
      }
    )
  }
}

比如，``brainstate.graph.nodes`` 可以轻松获取模型中涵盖的所有``Node``示例：

In [28]:
nodes = brainstate.graph.nodes(model)

nodes

{
  ('input',): Linear(
    in_size=(2,),
    out_size=(1,),
    w_mask=None,
    weight=ParamState(
      value={
        'bias': ShapedArray(float32[1]),
        'weight': ShapedArray(float32[2,1])
      }
    )
  ),
  ('layers', 0): Linear(
    in_size=(1,),
    out_size=(1,),
    w_mask=None,
    weight=ParamState(
      value={
        'bias': ShapedArray(float32[1]),
        'weight': ShapedArray(float32[1,1])
      }
    )
  ),
  ('layers', 1): Linear(
    in_size=(1,),
    out_size=(1,),
    w_mask=None,
    weight=ParamState(
      value={
        'bias': ShapedArray(float32[1]),
        'weight': ShapedArray(float32[1,1])
      }
    )
  ),
  ('layers', 2): Linear(
    in_size=(1,),
    out_size=(1,),
    w_mask=None,
    weight=ParamState(
      value={
        'bias': ShapedArray(float32[1]),
        'weight': ShapedArray(float32[1,1])
      }
    )
  ),
  ('output',): Linear(
    in_size=(1,),
    out_size=(3,),
    w_mask=None,
    weight=ParamState(
      value={
       

总的来说，``PyGraph`` 语法为科学计算中的复杂模型提供了强有力的支持，使用户能够更高效地构建、管理和优化计算图，从而推动脑动力学建模的研究和应用。

## 3. 程序编译

在高性能计算领域，硬件加速是提升计算效率的关键，而 ``BrainState`` 通过 ``State`` 语法实现了跨硬件的编译部署。``State`` 提供了一套高度抽象的接口，帮助用户编写一次代码便能生成程序中间表示（IR），并在不同硬件上进行编译和优化。

``brainstate``中提供的编译支持主要集成在 [``brainstate.compile`` 模块](../apis/compile.rst) 中。这些编译APIs囊括了一系列语法功能，包括：

- 条件语句： 支持 if-else 逻辑，方便用户根据不同条件执行不同的计算流程。
- 循环语句： 支持 for 循环，方便用户重复执行相同的计算操作。
- while 语句： 支持 while 循环，方便用户根据条件重复执行计算操作。
- 即时编译： 支持 JIT 即时编译，提高计算效率和性能。

brainstate编译的一大特色是，它只对``State``感知：在程序运行过程中，只要遇到一个``State``实例，就会将其编译进计算图，然后在不同硬件上运行。这种编译方式使得用户能够任意定义复杂的程序，而编译器会根据程序的实际运行分支进行针对性的优化，以此极大提高计算效率。同时，只对``State``感知的编译模式还使得用户能够更灵活地表达程序逻辑，而不用在意``PyGraph``、``PyTree``等概念的限制，从而彻底释放编程的灵活性。

以下是一个简单的编译示例：

In [29]:
a = brainstate.State(1.)


def add(i):
    a.value += 1.


brainstate.compile.for_loop(add, jnp.arange(10))

print(a.value)

11.0


在这个例子中，我们定义了一个简单的 for 循环，每次循环都会将 a 的值加 1。通过调用 bst.compile.for_loop 函数，我们将这个循环编译成计算图，并在 JAX 上运行。

brainstate编译的另一个特色是，它能嵌套地调用无论是JAX提供的函数式的编译函数还是brainstate内置的State感知的编译函数。中间步骤生成或利用的State变量将只会是局部变量，在整个程序中将被优化掉。这种特性使得程序内存占用更小，运行速度更快。

以下是一个简单的编译示例：

In [30]:
b = brainstate.State(0.)


def add(i):
    c = brainstate.State(0.)

    def cond(j):
        return j <= i

    def body(j):
        c.value += 1.
        return j + 1

    brainstate.compile.while_loop(cond, body, 0.)

    b.value += c.value


brainstate.compile.for_loop(add, jnp.arange(10))

print(b.value)

55.0


值得注意的是，brainstate 编译也支持使用 JAX 的调试工具进行调试。例如，用户可以通过调用 ``jax.debug.print`` 函数，打印出程序中间状态的值，方便调试和优化程序。以下示例是针对上面程序的一个调试输出。但更多关于 JAX 调试功能的信息，可以参考[JAX 调试文档](https://jax.readthedocs.io/en/latest/debugging/index.html)。

In [31]:
import jax

b = brainstate.State(0.)


def add(i):
    c = brainstate.State(0.)

    def cond(j):
        return j <= i

    def body(j):
        c.value += 1.
        return j + 1

    brainstate.compile.while_loop(cond, body, 0.)

    b.value += c.value
    jax.debug.print('b = {b}, c = {c}', b=b.value, c=c.value)


brainstate.compile.for_loop(add, jnp.arange(10))

b = 1.0, c = 1.0
b = 3.0, c = 2.0
b = 6.0, c = 3.0
b = 10.0, c = 4.0
b = 15.0, c = 5.0
b = 21.0, c = 6.0
b = 28.0, c = 7.0
b = 36.0, c = 8.0
b = 45.0, c = 9.0
b = 55.0, c = 10.0


brainstate还支持对不同硬件的编译。用户可以通过更改参数，将模型部署到不同的硬件上，包括 CPU、GPU 和 TPU。用户只需要在程序最开始调用：

```python
brainstate.environ.set(platform='cpu')  # CPU backend

brainstate.environ.set(platform='gpu')  # GPU backend

brainstate.environ.set(platform='tpu')  # TPU backend
```

或者使用jax的语法：

```python
jax.config.update('jax_platform_name', 'cpu')  # CPU backend

jax.config.update('jax_platform_name', 'gpu')  # GPU backend

jax.config.update('jax_platform_name', 'tpu')  # TPU backend
```

这种灵活的编译方式使得用户能够更好地利用不同硬件的优势，提高计算效率和性能。

## 4. 程序功能增强

brainstate还提供了一系列功能增强的转换。比如，虽然程序定义时只是前向推理，但是通过``grad``等自动微分转换，我们可以轻松地获得额外的梯度信息。这种功能增强的转换使得用户能够更方便地构建和优化复杂的计算模型。

但是，程序的功能增强需要提前知道程序的结构，需要已知我们需要增强的目标。因此，这就要求用户在编译之前就要知道程序的结构。为了这个目的，我们可以使用``PyGraph``语法，方便用户定义和管理计算模型。

``PyGraph``提供的关于``State``和图表示的各种操作和管理，极大地降低了我们构建各种复杂的程序功能增强转换的复杂度。brainstate中提供的已知的功能增强转换包括：

- 自动微分：自动求导的功能对模型优化至关重要，尤其在反向传播和梯度下降算法中。
- 批处理：支持大规模数据的批处理，有助于显著提升模型的训练速度和推理效率。
- 多设备并行：支持多设备并行计算，有助于提高模型的计算效率和性能。

以下是一个简单的自动微分示例：

In [32]:
# <input, output> pair
x = jnp.ones((1, 2))
y = jnp.ones((1, 3))

# model
model = brainstate.nn.Linear(2, 3)


# loss function
def loss_fn(x, y):
    return jnp.mean((y - model(x)) ** 2)


prev_loss = loss_fn(x, y)

# gradients
weights = model.states()
grads = brainstate.augment.grad(loss_fn, weights)(x, y)

# SGD update
for key, grad in grads.items():
    updates = jax.tree.map(lambda p, g: p - 0.1 * g, weights[key].value, grad)
    weights[key].value = updates

# loss evaluation
assert loss_fn(x, y) < prev_loss

在上面的例子中，我们定义了一个简单的线性模型，然后计算了模型的损失函数。通过调用 ``bst.augment.grad`` 函数，我们可以轻松地获取模型的梯度信息，并利用梯度下降算法对模型参数进行更新。但是，这种自动微分的功能增强转换，需要我们提前已知需要求梯度的参数是哪些，因此我们使用了 ``brainstate.graph.states`` 函数来获取模型中的所有 ``State`` 实例。

总的来说，程序功能增强转换为用户提供了一种更方便、更高效的方式来构建和优化计算模型。通过充分利用这些功能，用户能够更快地实现模型的训练和推理，提高模型的性能和效率。更多关于功能增强的转换，可以参考[程序功能增强教程](../tutorials/program_augmentation-zh.ipynb)。

## 5. 其它辅助功能

除了上述核心功能外，BrainState 还提供了许多辅助功能，帮助用户更便捷地进行模型构建和优化。这些功能包括但不限于：

- 随机数生成： 在模拟随机性或处理随机变量时可以快速生成分布不同的随机数。
- 参数管理： 提供简单的接口来初始化、存储和更新模型参数，适用于复杂的模型结构和多层网络。
- 调试工具： 帮助用户在模型开发过程中监控各层的状态和计算结果，便于发现潜在问题。

## 总结

BrainState 是一个功能强大的脑动力学建模框架，提供了跨硬件编译、计算模型增强、事件驱动计算和丰富的辅助工具。对于从事神经科学、认知建模和 SNN 开发的用户来说，BrainState 提供了丰富的模块化功能，支持用户快速构建、优化和部署高效的脑动力学模型。

通过充分理解和利用以上功能，您可以轻松创建和优化适用于不同研究任务和硬件平台的高效计算模型。